In [298]:
import dask.dataframe as dd
import dask.array as da
from dask.distributed import Client, TimeoutError, LocalCluster
import pandas as pd
import hvplot.dask
import re


#code found on stack exchange
#makes a new client if one is not running, runs 6 workers
try:
    client = Client('tcp://localhost:8787', timeout='2s')
except OSError or TimeoutError:
    cluster = LocalCluster(scheduler_port=8787, n_workers=6, ip='localhost')
    client = Client(cluster)
#

client


<Client: 'tcp://127.0.0.1:8787' processes=6 threads=12, memory=16.00 GiB>

In [299]:
keyWords = [
    'mental health', 
    'depression',
    'depressed', 
    'anxiety',
    'anxious', 
    'ADHD', 
    'attention deficit',
    'OCD',
    'obsessive-compulsive disorder',
    'PTSD',
    'posttraumatic stress disorder',
    'trauma',
    'bipolar',
    'autism',
    'schizophrenia',
    'eating disorder',
    'anorexia',
    'bulimia',
    'psychosis',
    'psychologist',
    'psychotherapist',
    'psychiatrist',
    'psychotherapy',
    'depressant'
]

In [300]:
#read and print the dataframes

#contains tweets (and time of the tweet) that contain mental health key word
#polled ever 1 minute
tweetdf = dd.read_csv(urlpath='mental_health_tweets.csv', sep='\t' )

#contains counts of tweets for every hour
countdf = dd.read_csv(urlpath='mental_health_tweet_count.csv', sep='\t')

print(tweetdf)

print(countdf)

Dask DataFrame Structure:
                tweet    time
npartitions=1                
               object  object
                  ...     ...
Dask Name: read-csv, 1 tasks
Dask DataFrame Structure:
              timestamp  count
npartitions=1                 
                 object  int64
                    ...    ...
Dask Name: read-csv, 1 tasks


In [301]:

#specify column types
tweetdf['time'] = dd.to_datetime(tweetdf['time'])
countdf['timestamp'] = dd.to_datetime(countdf['timestamp'])

#separate the components of the time for grouping
tweetdf['year'] = tweetdf['time'].dt.year
tweetdf['month'] = tweetdf['time'].dt.month
tweetdf['day'] = tweetdf['time'].dt.day
tweetdf['hour'] = tweetdf['time'].dt.hour
tweetdf['minute'] = tweetdf['time'].dt.minute

countdf['year'] = countdf['timestamp'].dt.year
countdf['month'] = countdf['timestamp'].dt.month
countdf['day'] = countdf['timestamp'].dt.day
countdf['hour'] = countdf['timestamp'].dt.hour

In [302]:
#sort the dataframes based on ascening tme
countdf = countdf.sort_values(ascending=True, by=['month', 'day', 'hour'])
tweetdf = tweetdf.sort_values(ascending=True, by=['month', 'day', 'hour', 'minute'])

#make a display string for the x axis of the graph
countdf['ts_display'] = countdf['month'].astype('str') + '/' + countdf['day'].astype('str') + '\t' + countdf['hour'].astype('str') + ':00'

#TODO MAKE THE GRAPH LOOK BETTER
countdf.hvplot(x='ts_display', y='count').opts(xrotation=90, fontscale=0.5)


:Curve   [ts_display]   (count)

In [303]:
#pre process

def preprocess(text):
    text = text[1]
    text = text.lower()
    return re.sub(string=text, pattern='[^a-z ]', repl='')

tweetdf['text'] = tweetdf['tweet'].map_partitions(preprocess)

In [304]:
#determine key words in tweet
def containsKey(text, key):
    return text.str.contains(key)

for i in keyWords:
    tweetdf[i] = tweetdf['tweet'].map_partitions(containsKey, i)


In [305]:
#count how many tweets each key appeared in
keyCount = pd.DataFrame(columns=keyWords)
keyCount = dd.from_pandas(keyCount, npartitions=1)

for i in keyWords:
    keyCount[i] = tweetdf[i].value_counts()

keyCount.compute()

,mental health,depression,depressed,anxiety,anxious,ADHD,attention deficit,OCD,obsessive-compulsive disorder,PTSD,...,schizophrenia,eating disorder,anorexia,bulimia,psychosis,psychologist,psychotherapist,psychiatrist,psychotherapy,depressant
False,41984,44764,53697,51902,53810,53539,53904,53820,53903,51491,...,53495,53486,53486,53738,53275,52997,53849,52967,53763,53749
True,11921,9141,208,2003,95,366,1,85,2,2414,...,410,419,419,167,630,908,56,938,142,156


In [ ]:
# tmp = pd.DataFrame(columns=keyWords)
# tmp = dd.from_pandas(tmp, chunksize=1000)

# for i in keyWords:
#     tmp = dd.concat([tmp, keyCount[i][1]], ignore_order=True)

# tmp.rename()

# tmp.compute()